#**RUTA OPTIMA**

In [62]:
import pandas as pd
from datetime import datetime, time, timedelta
from geopy.distance import geodesic
from collections import defaultdict

In [63]:

# ================= CONFIGURACIÓN INICIAL =================
COORDENADAS = {
    'AXE': (19.47150209787298, -99.20146505359766),
    'AMERICAN EAGLE': (19.467911152883392, -99.19827858921342),
    'Spotify': (19.46770884373848, -99.19571439741264),
    'TRAICIÓN by Dos Equis': (19.469090831714986, -99.20306092354295),
    'TRADICIÓN by Dos Equis': (19.469090831714986, -99.20306092354295),
    'TRAICIÓN "DOS EQUIS"': (19.469090831714986, -99.20306092354295)
}

In [64]:
# Calificaciones por persona (1-10)
calificaciones = {
    'Francisco': {
        'TYLER, THE CREATOR': 10, 'NSQK': 10, 'LIL YACHTY': 5, 'CAZRIEL & PACO AMOROSO': 5,
        'LEGALLYRXX': 1, 'TELESCREENS': 1, 'GESAFFELSTEIN': 1, 'MASSIVE ATTACK': 1,
        'FKA TWIGS': 9, 'NATHY PELUSO': 1, 'THE MARÍAS': 10, 'AGUSFORTNITE2008&&STIFFY': 7,
        'DAY2K': 1, 'RICHIE HAWTIN DEX EFX XOX': 1, 'BRUTALISMUS 3000': 3,
        'KELLY LEE OWENS': 1, 'ED MAVERICK': 1, 'FOUKERS': 1, 'NASH': 1, 'OG': 1,
        'BOBBY BEETHOVEN': 1, 'HORSEGIIRL': 7, 'PEPX ROMERO': 1, 'FOREPLAY': 1,
        'HETERA FRINÉ': 1,
        'NATANAEL CANO': 2, 'PARCELS': 7, 'MAGDALENA BAY': 9, 'MEME DEL REAL': 2,
        'JUAN CIREROL': 1, 'VALGUR': 1, 'SIMPSON AHUEVO': 1, "BARRY CAN'T SWIM LIVE": 1,
        'A. G. COOK': 6, 'THE DARE': 1, 'ARTEMAS': 1, 'YEYO': 1, 'PABLOPABLO': 1,
        'IZA TKM': 5, 'CHARLI XCX': 10, 'TOMORROW X TOGETHER': 5, 'HANUMANKIND': 1,
        'RALPHIE CHOO': 1, 'ARON': 1, 'LUISA ALMAGUER': 4, 'DJ GIGOLA': 1,
        'MAGNOLIA CORONADO': 4, 'PEPA ROMERO': 1, 'PIOLINDA MARCELA': 1,
        'ORLY ANAN': 5, 'DERRETIDA': 1
    },
    'Natalia': {
        'TYLER, THE CREATOR': 10, 'NSQK': 10, 'LIL YACHTY': 7, 'CAZRIEL & PACO AMOROSO': 10,
        'LEGALLYRXX': 1, 'TELESCREENS': 1, 'GESAFFELSTEIN': 1, 'MASSIVE ATTACK': 1,
        'FKA TWIGS': 10, 'NATHY PELUSO': 7, 'THE MARÍAS': 10, 'AGUSFORTNITE2008&&STIFFY': 1,
        'DAY2K': 1, 'RICHIE HAWTIN DEX EFX XOX': 1, 'BRUTALISMUS 3000': 1,
        'KELLY LEE OWENS': 1, 'ED MAVERICK': 7, 'FOUKERS': 1, 'NASH': 1, 'OG': 1,
        'BOBBY BEETHOVEN': 1, 'HORSEGIIRL': 1, 'PEPX ROMERO': 1, 'FOREPLAY': 1,
        'HETERA FRINÉ': 1,
        'NATANAEL CANO': 5, 'PARCELS': 5, 'MAGDALENA BAY': 10, 'MEME DEL REAL': 2,
        'JUAN CIREROL': 2, 'VALGUR': 2, 'SIMPSON AHUEVO': 1, "BARRY CAN'T SWIM LIVE": 2,
        'A. G. COOK': 5, 'THE DARE': 4, 'ARTEMAS': 2, 'YEYO': 2, 'PABLOPABLO': 2,
        'IZA TKM': 2, 'CHARLI XCX': 7, 'TOMORROW X TOGETHER': 9, 'HANUMANKIND': 2,
        'RALPHIE CHOO': 4, 'ARON': 2, 'LUISA ALMAGUER': 2, 'DJ GIGOLA': 2,
        'MAGNOLIA CORONADO': 2, 'PEPA ROMERO': 2, 'PIOLINDA MARCELA': 2,
        'ORLY ANAN': 2, 'DERRETIDA': 2
    },
    'Pedro': {
        'TYLER, THE CREATOR': 10, 'NSQK': 8, 'LIL YACHTY': 9, 'CAZRIEL & PACO AMOROSO': 8,
        'LEGALLYRXX': 4, 'TELESCREENS': 1, 'GESAFFELSTEIN': 5, 'MASSIVE ATTACK': 7,
        'FKA TWIGS': 8, 'NATHY PELUSO': 6, 'THE MARÍAS': 10, 'AGUSFORTNITE2008&&STIFFY': 5,
        'DAY2K': 3, 'RICHIE HAWTIN DEX EFX XOX': 4, 'BRUTALISMUS 3000': 6,
        'KELLY LEE OWENS': 5, 'ED MAVERICK': 7, 'FOUKERS': 7, 'NASH': 1, 'OG': 1,
        'BOBBY BEETHOVEN': 1, 'HORSEGIIRL': 4, 'PEPX ROMERO': 1, 'FOREPLAY': 4,
        'HETERA FRINÉ': 1,
        'NATANAEL CANO': 6, 'PARCELS': 7, 'MAGDALENA BAY': 10, 'MEME DEL REAL': 7,
        'JUAN CIREROL': 3, 'VALGUR': 7, 'SIMPSON AHUEVO': 7, "BARRY CAN'T SWIM LIVE": 2,
        'A. G. COOK': 3, 'THE DARE': 5, 'ARTEMAS': 1, 'YEYO': 4, 'PABLOPABLO': 5,
        'IZA TKM': 5, 'CHARLI XCX': 8, 'TOMORROW X TOGETHER': 4, 'HANUMANKIND': 1,
        'RALPHIE CHOO': 4, 'ARON': 1, 'LUISA ALMAGUER': 6, 'DJ GIGOLA': 4,
        'MAGNOLIA CORONADO': 1, 'PEPA ROMERO': 1, 'PIOLINDA MARCELA': 1,
        'ORLY ANAN': 1, 'DERRETIDA': 1
    },
    'Rudi': {
        'TYLER, THE CREATOR': 10, 'NSQK': 10, 'LIL YACHTY': 5, 'CAZRIEL & PACO AMOROSO': 10,
        'LEGALLYRXX': 1, 'TELESCREENS': 1, 'GESAFFELSTEIN': 2, 'MASSIVE ATTACK': 1,
        'FKA TWIGS': 10, 'NATHY PELUSO': 7, 'THE MARÍAS': 10, 'AGUSFORTNITE2008&&STIFFY': 6,
        'DAY2K': 1, 'RICHIE HAWTIN DEX EFX XOX': 1, 'BRUTALISMUS 3000': 1,
        'KELLY LEE OWENS': 1, 'ED MAVERICK': 10, 'FOUKERS': 1, 'NASH': 1, 'OG': 1,
        'BOBBY BEETHOVEN': 1, 'HORSEGIIRL': 1, 'PEPX ROMERO': 1, 'FOREPLAY': 1,
        'HETERA FRINÉ': 1,
        'NATANAEL CANO': 10, 'PARCELS': 1, 'MAGDALENA BAY': 10, 'MEME DEL REAL': 1,
        'JUAN CIREROL': 1, 'VALGUR': 1, 'SIMPSON AHUEVO': 9, "BARRY CAN'T SWIM LIVE": 1,
        'A. G. COOK': 1, 'THE DARE': 1, 'ARTEMAS': 1, 'YEYO': 1, 'PABLOPABLO': 1,
        'IZA TKM': 1, 'CHARLI XCX': 3, 'TOMORROW X TOGETHER': 1, 'HANUMANKIND': 1,
        'RALPHIE CHOO': 8, 'ARON': 1, 'LUISA ALMAGUER': 1, 'DJ GIGOLA': 1,
        'MAGNOLIA CORONADO': 1, 'PEPA ROMERO': 1, 'PIOLINDA MARCELA': 1,
        'ORLY ANAN': 1, 'DERRETIDA': 1
    },
    'Sebastian': {
        'TYLER, THE CREATOR': 8, 'NSQK': 10, 'LIL YACHTY': 5, 'CAZRIEL & PACO AMOROSO': 10,
        'LEGALLYRXX': 3, 'TELESCREENS': 2, 'GESAFFELSTEIN': 7, 'MASSIVE ATTACK': 3,
        'FKA TWIGS': 9, 'NATHY PELUSO': 9, 'THE MARÍAS': 9, 'AGUSFORTNITE2008&&STIFFY': 7,
        'DAY2K': 5, 'RICHIE HAWTIN DEX EFX XOX': 2, 'BRUTALISMUS 3000': 3,
        'KELLY LEE OWENS': 3, 'ED MAVERICK': 1, 'FOUKERS': 3, 'NASH': 1, 'OG': 1,
        'BOBBY BEETHOVEN': 3, 'HORSEGIIRL': 6, 'PEPX ROMERO': 4, 'FOREPLAY': 3,
        'HETERA FRINÉ': 3,
        'NATANAEL CANO': 7, 'PARCELS': 4, 'MAGDALENA BAY': 10, 'MEME DEL REAL': 2,
        'JUAN CIREROL': 1, 'VALGUR': 3, 'SIMPSON AHUEVO': 1, "BARRY CAN'T SWIM LIVE": 2,
        'A. G. COOK': 7, 'THE DARE': 3, 'ARTEMAS': 3, 'YEYO': 2, 'PABLOPABLO': 1,
        'IZA TKM': 1, 'CHARLI XCX': 9, 'TOMORROW X TOGETHER': 6, 'HANUMANKIND': 1,
        'RALPHIE CHOO': 8, 'ARON': 2, 'LUISA ALMAGUER': 1, 'DJ GIGOLA': 2,
        'MAGNOLIA CORONADO': 1, 'PEPA ROMERO': 2, 'PIOLINDA MARCELA': 1,
        'ORLY ANAN': 1, 'DERRETIDA': 2
    },
    'Carlos': {
        'TYLER, THE CREATOR': 8, 'NSQK': 3, 'LIL YACHTY': 5, 'CAZRIEL & PACO AMOROSO': 4,
        'LEGALLYRXX': 3, 'TELESCREENS': 2, 'GESAFFELSTEIN': 7, 'MASSIVE ATTACK': 10,
        'FKA TWIGS': 5, 'NATHY PELUSO': 5, 'THE MARÍAS': 4, 'AGUSFORTNITE2008&&STIFFY': 3,
        'DAY2K': 4, 'RICHIE HAWTIN DEX EFX XOX': 5, 'BRUTALISMUS 3000': 4,
        'KELLY LEE OWENS': 3, 'ED MAVERICK': 2, 'FOUKERS': 4, 'NASH': 2, 'OG': 1,
        'BOBBY BEETHOVEN': 4, 'HORSEGIIRL': 5, 'PEPX ROMERO': 2, 'FOREPLAY': 2,
        'HETERA FRINÉ': 2,
        'NATANAEL CANO': 10, 'PARCELS': 2, 'MAGDALENA BAY': 2, 'MEME DEL REAL': 2,
        'JUAN CIREROL': 3, 'VALGUR': 2, 'SIMPSON AHUEVO': 3, "BARRY CAN'T SWIM LIVE": 3,
        'A. G. COOK': 5, 'THE DARE': 3, 'ARTEMAS': 4, 'YEYO': 2, 'PABLOPABLO': 5,
        'IZA TKM': 3, 'CHARLI XCX': 7, 'TOMORROW X TOGETHER': 8, 'HANUMANKIND': 2,
        'RALPHIE CHOO': 2, 'ARON': 2, 'LUISA ALMAGUER': 2, 'DJ GIGOLA': 3,
        'MAGNOLIA CORONADO': 3, 'PEPA ROMERO': 2, 'PIOLINDA MARCELA': 2,
        'ORLY ANAN': 3, 'DERRETIDA': 3
    },
    'Emilio': {
        'TYLER, THE CREATOR': 10, 'NSQK': 4, 'LIL YACHTY': 8, 'CAZRIEL & PACO AMOROSO': 4,
        'LEGALLYRXX': 5, 'TELESCREENS': 1, 'GESAFFELSTEIN': 8, 'MASSIVE ATTACK': 4,
        'FKA TWIGS': 7, 'NATHY PELUSO': 7, 'THE MARÍAS': 10, 'AGUSFORTNITE2008&&STIFFY': 2,
        'DAY2K': 5, 'RICHIE HAWTIN DEX EFX XOX': 1, 'BRUTALISMUS 3000': 8,
        'KELLY LEE OWENS': 1, 'ED MAVERICK': 1, 'FOUKERS': 8, 'NASH': 1, 'OG': 1,
        'BOBBY BEETHOVEN': 1, 'HORSEGIIRL': 1, 'PEPX ROMERO': 1, 'FOREPLAY': 1,
        'HETERA FRINÉ': 1,
        'NATANAEL CANO': 8, 'PARCELS': 9, 'MAGDALENA BAY': 9, 'MEME DEL REAL': 5,
        'JUAN CIREROL': 6, 'VALGUR': 7, 'SIMPSON AHUEVO': 1, "BARRY CAN'T SWIM LIVE": 8,
        'A. G. COOK': 7, 'THE DARE': 6, 'ARTEMAS': 5, 'YEYO': 5, 'PABLOPABLO': 5,
        'IZA TKM': 5, 'CHARLI XCX': 10, 'TOMORROW X TOGETHER': 5, 'HANUMANKIND': 9,
        'RALPHIE CHOO': 5, 'ARON': 5, 'LUISA ALMAGUER': 5, 'DJ GIGOLA': 8,
        'MAGNOLIA CORONADO': 1, 'PEPA ROMERO': 1, 'PIOLINDA MARCELA': 1,
        'ORLY ANAN': 1, 'DERRETIDA': 1
    },
    'Rafa': {
        'TYLER, THE CREATOR': 10, 'NSQK': 7, 'LIL YACHTY': 8, 'CAZRIEL & PACO AMOROSO': 3,
        'LEGALLYRXX': 2, 'TELESCREENS': 2, 'GESAFFELSTEIN': 10, 'MASSIVE ATTACK': 6,
        'FKA TWIGS': 6, 'NATHY PELUSO': 6, 'THE MARÍAS': 9, 'AGUSFORTNITE2008&&STIFFY': 3,
        'DAY2K': 4, 'RICHIE HAWTIN DEX EFX XOX': 7, 'BRUTALISMUS 3000': 3,
        'KELLY LEE OWENS': 5, 'ED MAVERICK': 3, 'FOUKERS': 5, 'NASH': 4, 'OG': 1,
        'BOBBY BEETHOVEN': 4, 'HORSEGIIRL': 5, 'PEPX ROMERO': 4, 'FOREPLAY': 4,
        'HETERA FRINÉ': 4,
        'NATANAEL CANO': 1, 'PARCELS': 4, 'MAGDALENA BAY': 5, 'MEME DEL REAL': 5,
        'JUAN CIREROL': 5, 'VALGUR': 5, 'SIMPSON AHUEVO': 4, "BARRY CAN'T SWIM LIVE": 4,
        'A. G. COOK': 4, 'THE DARE': 8, 'ARTEMAS': 4, 'YEYO': 3, 'PABLOPABLO': 5,
        'IZA TKM': 5, 'CHARLI XCX': 10, 'TOMORROW X TOGETHER': 2, 'HANUMANKIND': 1,
        'RALPHIE CHOO': 5, 'ARON': 5, 'LUISA ALMAGUER': 3, 'DJ GIGOLA': 7,
        'MAGNOLIA CORONADO': 3, 'PEPA ROMERO': 3, 'PIOLINDA MARCELA': 3,
        'ORLY ANAN': 3, 'DERRETIDA': 3
    }
}

In [65]:
# Eventos por día (con horarios que pueden pasar de medianoche)
eventos_por_dia = {
    'sabado': [
        {'artista': 'NATANAEL CANO', 'inicio': '22:40', 'fin': '24:10', 'escenario': 'AXE'},
        {'artista': 'PARCELS', 'inicio': '20:25', 'fin': '21:40', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'MAGDALENA BAY', 'inicio': '18:30', 'fin': '19:30', 'escenario': 'Spotify'},
        {'artista': 'CHARLI XCX', 'inicio': '23:40', 'fin': '24:50', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'DJ GIGOLA', 'inicio': '24:30', 'fin': '26:00', 'escenario': 'TRAICIÓN by Dos Equis'},
        {'artista': 'SIMPSON AHUEVO', 'inicio': '24:40', 'fin': '25:40', 'escenario': 'Spotify'},
        {'artista': 'BARRY CAN\'T SWIM LIVE', 'inicio': '22:55', 'fin': '23:55', 'escenario': 'Spotify'},
        {'artista': 'A. G. COOK', 'inicio': '21:20', 'fin': '22:20', 'escenario': 'Spotify'},
        {'artista': 'THE DARE', 'inicio': '19:45', 'fin': '20:45', 'escenario': 'Spotify'},
        {'artista': 'ARTEMAS', 'inicio': '18:10', 'fin': '19:10', 'escenario': 'Spotify'},
        {'artista': 'YEYO', 'inicio': '16:45', 'fin': '17:35', 'escenario': 'Spotify'},
        {'artista': 'PABLOPABLO', 'inicio': '15:10', 'fin': '16:00', 'escenario': 'Spotify'},
        {'artista': 'IZA TKM', 'inicio': '14:10', 'fin': '14:40', 'escenario': 'Spotify'},
        {'artista': 'TOMORROW X TOGETHER', 'inicio': '21:30', 'fin': '22:40', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'HANUMANKIND', 'inicio': '19:30', 'fin': '20:30', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'RALPHIE CHOO', 'inicio': '17:40', 'fin': '18:40', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'ARON', 'inicio': '16:05', 'fin': '17:05', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'LUISA ALMAGUER', 'inicio': '14:40', 'fin': '15:25', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'MAGNOLIA CORONADO', 'inicio': '22:55', 'fin': '24:00', 'escenario': 'TRAICIÓN by Dos Equis'},
        {'artista': 'PEPA ROMERO', 'inicio': '20:50', 'fin': '22:20', 'escenario': 'TRAICIÓN by Dos Equis'},
        {'artista': 'PIOLINDA MARCELA', 'inicio': '20:20', 'fin': '20:40', 'escenario': 'TRAICIÓN by Dos Equis'},
        {'artista': 'ORLY ANAN', 'inicio': '18:40', 'fin': '20:10', 'escenario': 'TRAICIÓN by Dos Equis'},
        {'artista': 'DERRETIDA', 'inicio': '22:30', 'fin': '22:49', 'escenario': 'TRAICIÓN by Dos Equis'}
    ],
    'domingo': [
        {'artista': 'TYLER, THE CREATOR', 'inicio': '23:55', 'fin': '25:25', 'escenario': 'AXE'},
        {'artista': 'NSQK', 'inicio': '21:25', 'fin': '22:35', 'escenario': 'AXE'},
        {'artista': 'LIL YACHTY', 'inicio': '19:30', 'fin': '20:30', 'escenario': 'AXE'},
        {'artista': 'CAZRIEL & PACO AMOROSO', 'inicio': '17:45', 'fin': '18:45', 'escenario': 'AXE'},
        {'artista': 'LEGALLYRXX', 'inicio': '15:45', 'fin': '16:35', 'escenario': 'AXE'},
        {'artista': 'TELESCREENS', 'inicio': '14:30', 'fin': '15:20', 'escenario': 'AXE'},
        {'artista': 'GESAFFELSTEIN', 'inicio': '25:10', 'fin': '26:30', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'MASSIVE ATTACK', 'inicio': '22:05', 'fin': '23:55', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'FKA TWIGS', 'inicio': '20:15', 'fin': '21:15', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'NATHY PELUSO', 'inicio': '18:30', 'fin': '19:30', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'THE MARÍAS', 'inicio': '16:45', 'fin': '17:45', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'AGUSFORTNITE2008&&STIFFY', 'inicio': '15:05', 'fin': '15:55', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'DAY2K', 'inicio': '14:00', 'fin': '14:30', 'escenario': 'AMERICAN EAGLE'},
        {'artista': 'RICHIE HAWTIN DEX EFX XOX', 'inicio': '23:55', 'fin': '25:10', 'escenario': 'Spotify'},
        {'artista': 'BRUTALISMUS 3000', 'inicio': '20:50', 'fin': '22:05', 'escenario': 'Spotify'},
        {'artista': 'KELLY LEE OWENS', 'inicio': '19:25', 'fin': '20:15', 'escenario': 'Spotify'},
        {'artista': 'ED MAVERICK', 'inicio': '18:05', 'fin': '18:55', 'escenario': 'Spotify'},
        {'artista': 'FOUKERS', 'inicio': '16:25', 'fin': '17:15', 'escenario': 'Spotify'},
        {'artista': 'NASH', 'inicio': '15:10', 'fin': '15:40', 'escenario': 'Spotify'},
        {'artista': 'OG', 'inicio': '14:10', 'fin': '14:50', 'escenario': 'Spotify'},
        {'artista': 'BOBBY BEETHOVEN', 'inicio': '25:30', 'fin': '26:30', 'escenario': 'TRAICIÓN by Dos Equis'},
        {'artista': 'HORSEGIIRL', 'inicio': '23:55', 'fin': '25:25', 'escenario': 'TRAICIÓN by Dos Equis'},
        {'artista': 'PEPX ROMERO', 'inicio': '23:30', 'fin': '23:45', 'escenario': 'TRAICIÓN by Dos Equis'},
        {'artista': 'FOREPLAY', 'inicio': '22:25', 'fin': '23:15', 'escenario': 'TRAICIÓN by Dos Equis'},
        {'artista': 'HETERA FRINÉ', 'inicio': '21:45', 'fin': '22:05', 'escenario': 'TRAICIÓN by Dos Equis'}
    ]
}

In [66]:
def parse_horario(hora_str):
    "Parsea horas en formato HH:MM (acepta >=24) devolviendo (horas, minutos)"
    try:
        return tuple(map(int, hora_str.split(':')))  # (horas, minutos)
    except:
        return (0, 0)  # Fallback silencioso


scaling_factor = 0.5  # Example: reduce travel times by half

def generar_matriz_tiempos():
         return {
             esc1: {
                 esc2: 0 if esc1 == esc2 else max(1, int(geodesic(coord1, coord2).meters / 80 * scaling_factor))
                 for esc2, coord2 in COORDENADAS.items()
             }
             for esc1, coord1 in COORDENADAS.items()
         }

MATRIZ_TIEMPOS = generar_matriz_tiempos()

def generar_ruta_individual(dia, nombre_persona):
    "Genera ruta optimizada con manejo profesional de horarios extendidos"
    # 1. Configuración inicial
    fecha_base = datetime(2023, 4, 8) if dia.lower() == 'sábado' else datetime(2023, 4, 9)
    califs_persona = calificaciones.get(nombre_persona, {})
    artistas = []

    # 2. Procesamiento de eventos con manejo de horarios extendidos
    for evento in eventos_por_dia[dia]:
        artista = evento['artista']
        if artista not in califs_persona or califs_persona[artista] <= 0:
            continue

        # Parseo de horarios (pueden ser >24h)
        h_inicio, m_inicio = parse_horario(evento['inicio'])
        h_fin, m_fin = parse_horario(evento['fin'])

        # Cálculo de días adicionales y ajuste a formato 24h
        dias_extra_inicio = h_inicio // 24
        h_inicio = h_inicio % 24

        dias_extra_fin = h_fin // 24
        h_fin = h_fin % 24

        # Creación de objetos datetime con fecha correcta
        inicio_dt = datetime.combine(
            fecha_base.date() + timedelta(days=dias_extra_inicio),
            time(h_inicio, m_inicio)
        )
        fin_dt = datetime.combine(
            fecha_base.date() + timedelta(days=dias_extra_fin),
            time(h_fin, m_fin)
        )

        artistas.append({
            'nombre': artista,
            'inicio_dt': inicio_dt,
            'fin_dt': fin_dt,
            'escenario': evento['escenario'],
            'puntuacion': califs_persona[artista]
        })

    # 3. Ordenamiento cronológico
    artistas.sort(key=lambda x: x['inicio_dt'])

    # 4. Algoritmo de optimización (programación dinámica)
    n = len(artistas)
    dp = [a['puntuacion'] for a in artistas]
    prev = [-1] * n

    for i in range(n):
        for j in range(i):
            esc_j = artistas[j]['escenario']
            esc_i = artistas[i]['escenario']
            tiempo_desplaz = MATRIZ_TIEMPOS[esc_j][esc_i]
            tiempo_disponible = (artistas[i]['inicio_dt'] - artistas[j]['fin_dt']).total_seconds() / 60

            if tiempo_disponible >= tiempo_desplaz and dp[j] + artistas[i]['puntuacion'] > dp[i]:
                dp[i] = dp[j] + artistas[i]['puntuacion']
                prev[i] = j

    # 5. Reconstrucción de ruta óptima
    ruta = []
    if dp:
        max_idx = dp.index(max(dp))
        while max_idx != -1:
            ruta.append(artistas[max_idx])
            max_idx = prev[max_idx]
        ruta.reverse()

    # 6. Formateo de resultados
    resultados = []
    for i, evento in enumerate(ruta):
        desplazamiento = "Inicio" if i == 0 else \
            f"{int((evento['inicio_dt'] - ruta[i-1]['fin_dt']).total_seconds() / 60)} min"

        resultados.append({
            'Día': dia.capitalize(),
            'Persona': nombre_persona,
            'Artista': evento['nombre'],
            'Escenario': evento['escenario'],
            'Inicio': evento['inicio_dt'].strftime('%H:%M'),
            'Fin': evento['fin_dt'].strftime('%H:%M'),
            'Puntuación': evento['puntuacion'],
            'Desplazamiento': desplazamiento,
            'Puntuación Acumulada': sum(a['puntuacion'] for a in ruta[:i+1])
        })

    return pd.DataFrame(resultados)

#**Sabado**

In [51]:
# Ejemplo 1: Ruta de Francisco el sábado
print("\n" + "="*80)
print("RUTA INDIVIDUAL - Francisco (Sabado)")
print("="*80)
print(generar_ruta_individual('sabado', 'Francisco').to_string(index=False))

# Ejemplo 2: Ruta de Natalia el domingo
print("\n" + "="*80)
print("RUTA INDIVIDUAL - Natalia (Domingo)")
print("="*80)
print(generar_ruta_individual('domingo', 'Natalia').to_string(index=False))

# Ejemplo 3: Ruta de Pedro el sábado
print("\n" + "="*80)
print("RUTA INDIVIDUAL - Pedro (Sabado)")
print("="*80)
print(generar_ruta_individual('sabado', 'Pedro').to_string(index=False))

# Ejemplo 4: Ruta de Rudi el domingo
print("\n" + "="*80)
print("RUTA INDIVIDUAL - Rudi (Domingo)")
print("="*80)
print(generar_ruta_individual('domingo', 'Rudi').to_string(index=False))

# Ejemplo 5: Ruta de Sebastian el sábado
print("\n" + "="*80)
print("RUTA INDIVIDUAL - Sebastian (Sabado)")
print("="*80)
print(generar_ruta_individual('sabado', 'Sebastian').to_string(index=False))

# Ejemplo 6: Ruta de Carlos el domingo
print("\n" + "="*80)
print("RUTA INDIVIDUAL - Carlos (Domingo)")
print("="*80)
print(generar_ruta_individual('domingo', 'Carlos').to_string(index=False))

# Ejemplo 7: Ruta de Emilio el sábado
print("\n" + "="*80)
print("RUTA INDIVIDUAL - Emilio (Sabado)")
print("="*80)
print(generar_ruta_individual('sabado', 'Emilio').to_string(index=False))


RUTA INDIVIDUAL - Francisco (Sabado)
   Día   Persona       Artista             Escenario Inicio   Fin  Puntuación Desplazamiento  Puntuación Acumulada
Sabado Francisco       IZA TKM               Spotify  14:10 14:40           5         Inicio                     5
Sabado Francisco    PABLOPABLO               Spotify  15:10 16:00           1         30 min                     6
Sabado Francisco          ARON        AMERICAN EAGLE  16:05 17:05           1          5 min                     7
Sabado Francisco MAGDALENA BAY               Spotify  18:30 19:30           9         85 min                    16
Sabado Francisco       PARCELS        AMERICAN EAGLE  20:25 21:40           7         55 min                    23
Sabado Francisco     DERRETIDA TRAICIÓN by Dos Equis  22:30 22:49           1         50 min                    24
Sabado Francisco    CHARLI XCX        AMERICAN EAGLE  23:40 00:50          10         51 min                    34

RUTA INDIVIDUAL - Natalia (Domingo)
    D

In [67]:
def generar_timelines_sabado(calificaciones, eventos_por_dia, COORDENADAS):
    """Genera timelines para todas las personas el sabado"""

    # Configuración base
    fecha_base = datetime(2023, 4, 8)
    dia = 'sabado'

    # Procesar cada persona
    for nombre_persona in calificaciones:
        print("\n" + "="*80)
        print(f"RUTA PARA {nombre_persona.upper()} - SABADO")
        print("="*80)

        # Filtrar artistas con puntuación > 0
        artistas = []
        for evento in eventos_por_dia[dia]:
            artista = evento['artista']
            if artista in calificaciones[nombre_persona] and calificaciones[nombre_persona][artista] > 0:
                # Parsear horas extendidas
                h_inicio, m_inicio = map(int, evento['inicio'].split(':'))
                h_fin, m_fin = map(int, evento['fin'].split(':'))

                dias_extra_inicio = h_inicio // 24
                dias_extra_fin = h_fin // 24

                inicio_dt = datetime.combine(
                    fecha_base.date() + timedelta(days=dias_extra_inicio),
                    time(h_inicio % 24, m_inicio)
                )
                fin_dt = datetime.combine(
                    fecha_base.date() + timedelta(days=dias_extra_fin),
                    time(h_fin % 24, m_fin)
                )

                artistas.append({
                    'nombre': artista,
                    'inicio_dt': inicio_dt,
                    'fin_dt': fin_dt,
                    'escenario': evento['escenario'],
                    'puntuacion': calificaciones[nombre_persona][artista]
                })

        # Ordenar cronológicamente
        artistas.sort(key=lambda x: x['inicio_dt'])

        # Algoritmo de optimización
        n = len(artistas)
        dp = [a['puntuacion'] for a in artistas]
        prev = [-1] * n

        # Generar matriz de tiempos
        MATRIZ_TIEMPOS = {
            esc1: {esc2: max(1, int(geodesic(COORDENADAS[esc1], COORDENADAS[esc2]).meters / 80))
                  for esc2 in COORDENADAS}
            for esc1 in COORDENADAS
        }

        for i in range(n):
            for j in range(i):
                tiempo_desplaz = MATRIZ_TIEMPOS[artistas[j]['escenario']][artistas[i]['escenario']]
                tiempo_disponible = (artistas[i]['inicio_dt'] - artistas[j]['fin_dt']).total_seconds() / 60

                if tiempo_disponible >= tiempo_desplaz and dp[j] + artistas[i]['puntuacion'] > dp[i]:
                    dp[i] = dp[j] + artistas[i]['puntuacion']
                    prev[i] = j

        # Reconstruir ruta óptima
        ruta = []
        if dp:
            max_idx = dp.index(max(dp))
            while max_idx != -1:
                ruta.append(artistas[max_idx])
                max_idx = prev[max_idx]
            ruta.reverse()

        # Mostrar timeline
        if not ruta:
            print("No hay eventos con puntuación positiva para esta persona")
            continue

        print(f"Puntuación total: {sum(a['puntuacion'] for a in ruta)}")

        for i, evento in enumerate(ruta):
            # Determinar si es del día siguiente
            dias_extra = (evento['inicio_dt'].date() - fecha_base.date()).days
            dia_prefix = f"+{dias_extra}d " if dias_extra > 0 else ""

            # Mostrar evento
            print(f"\n[{dia_prefix}{evento['inicio_dt'].strftime('%H:%M')} - {evento['fin_dt'].strftime('%H:%M')}]")
            print(f"Artista: {evento['nombre']}")
            print(f"Escenario: {evento['escenario']}")
            print(f"Puntuación: {evento['puntuacion']}")

            # Mostrar desplazamiento si no es el primer evento
            if i > 0:
                tiempo = (evento['inicio_dt'] - ruta[i-1]['fin_dt']).total_seconds() / 60
                req = MATRIZ_TIEMPOS[ruta[i-1]['escenario']][evento['escenario']]
                print(f"Tiempo desde el anterior: {int(tiempo)} min (requeridos: {req} min)")

        print("\n" + "-"*60)

# Ejecutar para todas las personas
generar_timelines_sabado(calificaciones, eventos_por_dia, COORDENADAS)


RUTA PARA FRANCISCO - SABADO
Puntuación total: 34

[14:10 - 14:40]
Artista: IZA TKM
Escenario: Spotify
Puntuación: 5

[15:10 - 16:00]
Artista: PABLOPABLO
Escenario: Spotify
Puntuación: 1
Tiempo desde el anterior: 30 min (requeridos: 1 min)

[16:05 - 17:05]
Artista: ARON
Escenario: AMERICAN EAGLE
Puntuación: 1
Tiempo desde el anterior: 5 min (requeridos: 3 min)

[18:30 - 19:30]
Artista: MAGDALENA BAY
Escenario: Spotify
Puntuación: 9
Tiempo desde el anterior: 85 min (requeridos: 3 min)

[20:25 - 21:40]
Artista: PARCELS
Escenario: AMERICAN EAGLE
Puntuación: 7
Tiempo desde el anterior: 55 min (requeridos: 3 min)

[22:30 - 22:49]
Artista: DERRETIDA
Escenario: TRAICIÓN by Dos Equis
Puntuación: 1
Tiempo desde el anterior: 50 min (requeridos: 6 min)

[23:40 - 00:50]
Artista: CHARLI XCX
Escenario: AMERICAN EAGLE
Puntuación: 10
Tiempo desde el anterior: 51 min (requeridos: 6 min)

------------------------------------------------------------

RUTA PARA NATALIA - SABADO
Puntuación total: 36

[14:

#**Domingo**

In [68]:
def generar_timelines_domingo(calificaciones, eventos_por_dia, COORDENADAS):
    """Genera timelines para todas las personas el domingo"""

    # Configuración base
    fecha_base = datetime(2023, 4, 9)  # Fecha del domingo
    dia = 'domingo'

    # Procesar cada persona
    for nombre_persona in calificaciones:
        print("\n" + "="*80)
        print(f"RUTA PARA {nombre_persona.upper()} - DOMINGO")
        print("="*80)

        # Filtrar artistas con puntuación > 0
        artistas = []
        for evento in eventos_por_dia[dia]:
            artista = evento['artista']
            if artista in calificaciones[nombre_persona] and calificaciones[nombre_persona][artista] > 0:
                # Parsear horas extendidas
                h_inicio, m_inicio = map(int, evento['inicio'].split(':'))
                h_fin, m_fin = map(int, evento['fin'].split(':'))

                dias_extra_inicio = h_inicio // 24
                dias_extra_fin = h_fin // 24

                inicio_dt = datetime.combine(
                    fecha_base.date() + timedelta(days=dias_extra_inicio),
                    time(h_inicio % 24, m_inicio)
                )
                fin_dt = datetime.combine(
                    fecha_base.date() + timedelta(days=dias_extra_fin),
                    time(h_fin % 24, m_fin)
                )

                artistas.append({
                    'nombre': artista,
                    'inicio_dt': inicio_dt,
                    'fin_dt': fin_dt,
                    'escenario': evento['escenario'],
                    'puntuacion': calificaciones[nombre_persona][artista]
                })

        # Ordenar cronológicamente
        artistas.sort(key=lambda x: x['inicio_dt'])

        # Algoritmo de optimización
        n = len(artistas)
        dp = [a['puntuacion'] for a in artistas]
        prev = [-1] * n

        # Generar matriz de tiempos
        MATRIZ_TIEMPOS = {
            esc1: {esc2: max(1, int(geodesic(COORDENADAS[esc1], COORDENADAS[esc2]).meters / 80))
                  for esc2 in COORDENADAS}
            for esc1 in COORDENADAS
        }

        for i in range(n):
            for j in range(i):
                tiempo_desplaz = MATRIZ_TIEMPOS[artistas[j]['escenario']][artistas[i]['escenario']]
                tiempo_disponible = (artistas[i]['inicio_dt'] - artistas[j]['fin_dt']).total_seconds() / 60

                if tiempo_disponible >= tiempo_desplaz and dp[j] + artistas[i]['puntuacion'] > dp[i]:
                    dp[i] = dp[j] + artistas[i]['puntuacion']
                    prev[i] = j

        # Reconstruir ruta óptima
        ruta = []
        if dp:
            max_idx = dp.index(max(dp))
            while max_idx != -1:
                ruta.append(artistas[max_idx])
                max_idx = prev[max_idx]
            ruta.reverse()

        # Mostrar timeline
        if not ruta:
            print("No hay eventos con puntuación positiva para esta persona")
            continue

        print(f"Puntuación total: {sum(a['puntuacion'] for a in ruta)}")

        for i, evento in enumerate(ruta):
            # Determinar si es del día siguiente
            dias_extra = (evento['inicio_dt'].date() - fecha_base.date()).days
            dia_prefix = f"+{dias_extra}d " if dias_extra > 0 else ""

            # Mostrar evento
            print(f"\n[{dia_prefix}{evento['inicio_dt'].strftime('%H:%M')} - {evento['fin_dt'].strftime('%H:%M')}]")
            print(f"Artista: {evento['nombre']}")
            print(f"Escenario: {evento['escenario']}")
            print(f"Puntuación: {evento['puntuacion']}")

            # Mostrar desplazamiento si no es el primer evento
            if i > 0:
                tiempo = (evento['inicio_dt'] - ruta[i-1]['fin_dt']).total_seconds() / 60
                req = MATRIZ_TIEMPOS[ruta[i-1]['escenario']][evento['escenario']]
                print(f"Tiempo desde el anterior: {int(tiempo)} min (requeridos: {req} min)")

        print("\n" + "-"*60)

# Ejecutar para todas las personas
generar_timelines_domingo(calificaciones, eventos_por_dia, COORDENADAS)


RUTA PARA FRANCISCO - DOMINGO
Puntuación total: 50

[14:00 - 14:30]
Artista: DAY2K
Escenario: AMERICAN EAGLE
Puntuación: 1

[15:05 - 15:55]
Artista: AGUSFORTNITE2008&&STIFFY
Escenario: AMERICAN EAGLE
Puntuación: 7
Tiempo desde el anterior: 35 min (requeridos: 1 min)

[16:45 - 17:45]
Artista: THE MARÍAS
Escenario: AMERICAN EAGLE
Puntuación: 10
Tiempo desde el anterior: 50 min (requeridos: 1 min)

[18:05 - 18:55]
Artista: ED MAVERICK
Escenario: Spotify
Puntuación: 1
Tiempo desde el anterior: 20 min (requeridos: 3 min)

[20:15 - 21:15]
Artista: FKA TWIGS
Escenario: AMERICAN EAGLE
Puntuación: 9
Tiempo desde el anterior: 80 min (requeridos: 3 min)

[21:25 - 22:35]
Artista: NSQK
Escenario: AXE
Puntuación: 10
Tiempo desde el anterior: 10 min (requeridos: 6 min)

[23:30 - 23:45]
Artista: PEPX ROMERO
Escenario: TRAICIÓN by Dos Equis
Puntuación: 1
Tiempo desde el anterior: 55 min (requeridos: 3 min)

[23:55 - 01:25]
Artista: TYLER, THE CREATOR
Escenario: AXE
Puntuación: 10
Tiempo desde el anter